In [1]:
import os
from os.path import join
import sys
import torch
from torch import nn
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from scipy.sparse import csr_matrix
import scipy.sparse as sp
import time
import dgl
import dgl.function as fn
import mxnet as mx
import torch.nn as nn

Using backend: pytorch


In [2]:
def dataload():
    train_file = "./gowalla/train.txt"
    test_file = "./gowalla/test.txt"
    m_item = 0
    n_user = 0
    trainUniqueUsers, trainItem, trainUser = [], [], []
    testUniqueUsers, testItem, testUser = [], [], []
    with open(train_file) as f:
        for l in f.readlines():
            if len(l) > 0:
                l = l.strip('\n').split(' ')
                items = [int(i) for i in l[1:]]
                uid = int(l[0])
                trainUniqueUsers.append(uid)
                trainUser.extend([uid] * len(items))
                trainItem.extend(items)
                m_item = max(m_item, max(items))
                n_user = max(n_user, uid)
    trainUser = np.array(trainUser)
    trainItem = np.array(trainItem)
    return trainUser,trainItem

In [3]:
def getgraph(trainUser,trainItem):
    g = dgl.heterograph({('user', 'like' , 'item'): (trainUser, trainItem),
                         ('item', 'good', 'user'): (trainItem,trainUser)})
    gre_user = g.out_degrees(etype=('user', 'like' , 'item'))
    gre_item = g.out_degrees(etype=('item', 'good' , 'user'))
    gre_user = gre_user ** (-0.5)
    gre_user = gre_user.unsqueeze(1).repeat(1,64)
    gre_item = gre_item  ** (-0.5)
    gre_item = gre_item.unsqueeze(1).repeat(1,64)
    return g, gre_user, gre_item

In [4]:
trainUser,trainItem= dataload()
g, gre_user, gre_item= getgraph(trainUser,trainItem)

In [1]:
class Lightgcn(nn.Module):
    def __init__(self,
                g,
                gre_user,
                gre_item,
                n_layers):
        super(Lightgcn, self).__init__()
        self.g = g
        self.gre_user = gre_user
        self.gre_item = gre_item
        self.layers = n_layers
        self.num_users = g.num_nodes('user')
        self.num_items = g.num_nodes('item')
        self.__init_weight()
    def __init_weight(self):
        self.embedding_user = torch.nn.Embedding(
            num_embeddings=self.num_users, embedding_dim=self.latent_dim)
        self.embedding_item = torch.nn.Embedding(
            num_embeddings=self.num_items, embedding_dim=self.latent_dim)
    
    def computer(self):
        User = []
        Item = []
        U.append(g.nodes['user'].data['emb'])
        I.append(g.nodes['item'].data['emb'])
        for i in range(self.layers):
            g.nodes['user'].data['emb'] = torch.mul(g.nodes['user'].data['emb'], self.gre_user)
            g.nodes['item'].data['emb'] = torch.mul(g.nodes['item'].data['emb'], self.gre_item)
            g.multi_update_all(
                {'edg1':(fn.copy_src('emb', 'm'), fn.sum('m', 'emb')),
                'edg2':(fn.copy_src('emb', 'm'), fn.sum('m', 'emb'))},
                "sum"
            )
            g.nodes['user'].data['emb'] = torch.mul(g.nodes['user'].data['emb'], gre_user)
            g.nodes['item'].data['emb'] = torch.mul(g.nodes['item'].data['emb'], gre_item)
            U.append(g.nodes['user'].data['emb'])
            I.append(g.nodes['item'].data['emb'])
        out_user = torch.stack(U,dim=1)
        out_user = torch.mean(out_user,dim=1)
        out_item = torch.stack(I,dim=1)
        out_item = torch.mean(out_item,dim=1)
        return out_user, out_item
    
    def bpr_loss(self)
        out_user, out_item = self.computer()
        score = torch.mm(out_user, out_item.t())
        
    
        
        
        
    
    

SyntaxError: invalid syntax (<ipython-input-1-85b96e9d65e7>, line 44)

In [ ]:
def UniformSample_original(trainUser,trainItem, g):
    n_user = g.num_nodes('user')
    m_item = g.num_nodes('item')
    UserItemNet = csr_matrix((np.ones(len(trainUser)), (trainUser, trainItem)),
                                      shape=(n_user, m_item))
    total_start = time()
    user_num = len(trainUser)
    users = np.random.randint(0, n_user, user_num)
    S = []
    sample_time1 = 0.
    sample_time2 = 0.
    allPos = []
    for user in range(n_user):
        allPos.append(UserItemNet[user].nonzero()[1])
    for i, user in enumerate(users):
        start = time()
        posForUser = allPos[user]
        if len(posForUser) == 0:
            continue
        sample_time2 += time() - start
        posindex = np.random.randint(0, len(posForUser))
        positem = posForUser[posindex]
        while True:
            negitem = np.random.randint(0, m_item)
            if negitem in posForUser:
                continue
            else:
                break
        S.append([user, positem, negitem])
        end = time()
        sample_time1 += end - start
    total = time() - total_start
    return np.array(S), [total, sample_time1, sample_time2]
    